In [77]:
# load dataset
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import simplejson as json

df = pd.read_csv("/users/akuppam/documents/Hprog/Py/rnb1015_2.csv")
df.head(5)

,date,region,marketing,visits,br,inq,gb,cb,nb,ss,ts,listings
0,1/1/16,AMR,NotPaid,1256288,6043,66022,6071,571,5500,485968.08000,144695.894200,10242216
1,1/1/16,AMR,Paid,189297,1092,11806,743,2,741,0.00000,14138.809600,2133795
2,1/1/16,UK,NotPaid,201797,435,9662,436,33,403,72698.92472,8317.105056,3561696
3,1/1/16,UK,Paid,55934,88,2749,61,1,60,0.00000,815.336426,890424
4,1/1/16,CE,NotPaid,200786,573,9250,435,31,404,40490.73159,12016.782410,3062480


In [78]:
df = pd.read_csv("/users/akuppam/documents/Hprog/Py/rnb1015_2.csv")
df = df.loc[(df['region'] == 'AMR') & (df['marketing'] == 'NotPaid')]
df = df[~(df['nb'] < 0)]
df.describe()
df.index = pd.to_datetime(df.date)
cols = ["nb"]
df = df[cols]
df.columns = ['y']
df.head()

,y
date,
2016-01-01,5500
2016-01-02,6513
2016-01-03,7602
2016-01-04,8871
2016-01-05,9828


In [72]:
import warnings
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	history = [x for x in X]
	# make predictions
	predictions = list()
	for t in range(len(X)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(transparams=True)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(X[t])
	# calculate out of sample error
	error = mean_squared_error(X, predictions)
	mape = np.mean(abs((X-predictions)/X))
	return error
	return mape

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_params = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_params = mse, order
					print('ARIMA %s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA %s MSE=%.3f' % (best_params, best_score))
	#model_params = [p,d,q]
	print("best order =", model_params)
	with open('model_params.json','w') as out_params:
		json.dump(best_params, out_params)

# load dataset
df.index = pd.to_datetime(df.date)
cols = ["nb"]
y = df[cols]
series = y['nb']

# evaluate parameters
p_values = range(0, 13)
d_values = range(0, 2)
q_values = range(0, 13)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

# select the best set of p,d,q
with open('model_params.json') as json_file:
    out_params = json.load(json_file)
out_params
p = out_params[0]
d = out_params[1]
q = out_params[2]

# apply model
model = ARIMA(series, order=(p,d,q))
model_fit = model.fit(transparams=True)
print(model_fit.summary())

# Store predictions
import numpy as np
X = series.values
history = [x for x in X]
predictions = list()
for t in range(len(X)):
	model = ARIMA(history, order=(p,d,q))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = X[t]
	history.append(obs)

# Get forecast 442 steps ahead in future (10/15/2018 to 12/31/2019)
yhat = model_fit.forecast(steps=442)
series_arima = predictions.append(yhat)

AttributeError: 'DataFrame' object has no attribute 'date'

In [85]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def ArimaFxn(df, fcstdays):
    
    series = df['y']
    # evaluate parameters
    p_values = range(5, 6)
    d_values = range(1, 2)
    q_values = range(5, 6)
    
    best_score, best_params = float("inf"), None
    series = series.astype('double')
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                mse = evaluate_arima_model(series, order, fcstdays)
                if mse < best_score:
                    best_score, best_params = mse, order
                    print('ARIMA MSE=%.3f' % (mse))
                    print('Best ARIMA %s MSE=%.3f' % (best_params, best_score))
                    print('Best Params: ', best_params)
                    
    # apply model
    model = ARIMA(series, order=(best_params[0],best_params[1],best_params[2]))
    model_fit = model.fit(transparams=True)
    print(model_fit.summary())
    
    pred_fcst = model_fit.predict(start=1, end=len(df['y'])-1, dynamic=False)
    #pf = pred_fcst.values
    arima_forecast = model_fit.forecast(fcstdays)[0]
    af = pd.Series(arima_forecast)
    arima_series = pred_fcst.append(af)
    arima_series.to_csv('arima_series.csv')
    
    return arima_series
    #return pred_fcst, arima_forecast
    #return pred_fcst
    #return arima_forecast[0]
    #return af


    
def evaluate_arima_model(X, order, fcstdays):
    # make predictions
    predictions = list()
    model = ARIMA(X,order=order)
    model_fit = model.fit(transparams=True)
    predictions = model_fit.predict(start=1, end=len(X), dynamic=False)
    yhat = model_fit.forecast(fcstdays)    
    # calculate out of sample error
    error = mean_squared_error(X, predictions)
    print('error = ', error)
    return error

In [7]:
# pred_fcst (.predict) should return 'array'
# arima_forecast (.forecast) should also return 'array'

In [86]:
ArimaFxn(df, 365)

error =  87481719.66616653
ARIMA MSE=87481719.666
Best ARIMA (5, 1, 5) MSE=87481719.666
Best Params:  (5, 1, 5)
                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                 1018
Model:                 ARIMA(5, 1, 5)   Log Likelihood               -8144.231
Method:                       css-mle   S.D. of innovations            719.114
Date:                Tue, 20 Nov 2018   AIC                          16312.462
Time:                        14:38:55   BIC                          16371.569
Sample:                    01-02-2016   HQIC                         16334.907
                         - 10-15-2018                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0241     11.703     -0.002      0.998     -22.962      22.914
ar.L1.D.y      0.86

2016-01-02 00:00:00      -0.024117
2016-01-03 00:00:00     -11.178920
2016-01-04 00:00:00    -230.445477
2016-01-05 00:00:00    -540.981063
2016-01-06 00:00:00   -1023.589140
2016-01-07 00:00:00   -1638.712499
2016-01-08 00:00:00   -1630.687686
2016-01-09 00:00:00    -493.357746
2016-01-10 00:00:00     601.314649
2016-01-11 00:00:00     587.662840
2016-01-12 00:00:00     331.707040
2016-01-13 00:00:00     121.253494
2016-01-14 00:00:00    -454.213550
2016-01-15 00:00:00    -856.832430
2016-01-16 00:00:00    -253.090821
2016-01-17 00:00:00     836.320602
2016-01-18 00:00:00     746.581759
2016-01-19 00:00:00     141.233925
2016-01-20 00:00:00      79.965401
2016-01-21 00:00:00    -330.709103
2016-01-22 00:00:00    -938.841189
2016-01-23 00:00:00    -282.506556
2016-01-24 00:00:00     685.412473
2016-01-25 00:00:00     746.317918
2016-01-26 00:00:00       9.711574
2016-01-27 00:00:00    -171.696346
2016-01-28 00:00:00    -384.289899
2016-01-29 00:00:00    -631.595283
2016-01-30 00:00:00 

In [ ]:
plt.plot(series)
plt.plot(series_arima, color='blue')
plt.plot(model_fit.fittedvalues, color='red')